In [ ]:
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 40.4 MB/s eta 0:00:00


In [ ]:
from numpy import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary
from torchvision import datasets
from torchvision import transforms
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from mealpy.swarm_based.AVOA import OriginalAVOA
from mealpy.utils.problem import Problem
from tqdm import tqdm
from mealpy import FloatVar

In [ ]:
class MNISTCNN(nn.Module):
    """ConvNet -> Max_Pool -> RELU -> ConvNet ->
    Max_Pool -> RELU -> FC -> RELU -> FC -> SOFTMAX"""
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)



    def forward(self, x):

        # Reshape input for convolutional layers
        x = x.view(-1, 1, 28, 28)  # Assuming MNIST images are 28x28 pixels with a single channel
        x = F.relu(self.conv1(x.float()))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [ ]:
#loss functions
def loss_classifier(pred,labels):

  loss = nn.CrossEntropyLoss()

  return loss(pred,labels)



In [ ]:
#

In [ ]:
#loading mnist and spliting it in clients
from torch.utils.data import DataLoader

def create_loaders(client_datasets, batch_size, shuffle):
    loaders = []

    for client_data in client_datasets:
        x_data, y_data = client_data
        dataset = list(zip(x_data, y_data))
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        loaders.append(loader)

    return loaders


def iid_split(x_train, y_train, x_test, y_test, n_clients, n_samples_train, n_samples_test, batch_size, shuffle=True):
    client_datasets_train, client_datasets_test = [], []

    for _ in range(n_clients):
        client_x_train, _, client_y_train, _ = train_test_split(x_train, y_train, train_size=n_samples_train, stratify=y_train, shuffle=shuffle)
        client_x_test, _, client_y_test, _ = train_test_split(x_test, y_test, train_size=n_samples_test, stratify=y_test, shuffle=shuffle)

        client_datasets_train.append((client_x_train, client_y_train))
        client_datasets_test.append((client_x_test, client_y_test))

    train = create_loaders(client_datasets_train, batch_size, shuffle)
    test = create_loaders(client_datasets_test, batch_size, shuffle)

    return train, test



def MNIST(n_samples_train = 200, n_samples_test = 100, n_clients = 3, batch_size = 25, shuffle = True):


  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  x_train = x_train.reshape((len(x_train), -1)) / 255.0
  x_test = x_test.reshape((len(x_test), -1)) / 255.0

  train, test=iid_split(x_train, y_train, x_test, y_test, n_clients, n_samples_train, n_samples_test, batch_size, shuffle)

  return train,test

In [ ]:
#Genric functions
def loss_dataset(model,dl,loss):
  loss_num = 0

  for id,(data,target) in enumerate(dl):
      predictions = model(data)
      loss_num+=loss(predictions,target)

  loss_num/=id+1
  return loss_num

def accuracy_dataset(model,dataset):

  correct = 0

  for id,(data,target) in enumerate(dataset):
    predictions = model(data)

    _,predicted = predictions.max(1,keepdim=True)
    print(predicted)
    print(predicted.view(-1,1))
    print(target.view(-1,1).item())
    correct+=torch.sum(predicted.view(-1,1)==target.view(-1,1).item())

  accuracy = 100*correct/len(data)

  return accuracy

def train_step(model,optimizer,train_data,loss_f):

  total_loss = 0

  for idx, (features,labels) in enumerate(train_data):

    optimizer.zero_grad()
    predictions = model(features)
    loss = loss_f(predictions,labels)
    total_loss += loss

    loss.backward()

    optimizer.step()

  return total_loss

def local_learning(model,optimizer,train,local_epoch,loss_f):

  for e in range(local_epoch):

    local_loss = train_step(model,optimizer,train,loss_f)

    return float(local_loss.detach().cpu().numpy())


In [ ]:
def FEDAVO(train,test,model,n_iter,local_epoch,tuning_epoch):

    loss_f=loss_classifier
    no_clients = len(train)
    n_samples=sum([len(db.dataset) for db in train])
    weights=([len(db.dataset)/n_samples for db in train])
    print("Clients' weights:",weights)



    loss_hist=[[float(loss_dataset(model,dl, loss_f).detach()) for dl in train]]
    acc_hist=[[accuracy_dataset(model, dl) for dl in test]]
    print(acc_hist)
    server_hist=[[tens_param.detach().cpu().numpy() for tens_param in list(model.parameters())]]

    models_hist = []

    server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))]) #sum the loss and accuracy of all the three clients
    server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])

    print(f'====> i: 0 Loss: {server_loss}')

    server_loss_list = [] #list to store the sum client of server loss and accuracy
    server_accuracy_list = []

    for i in tqdm(range(n_iter)):

        clients_params=[]
        clients_models=[]
        clients_losses=[]

        clients_param=[]
        clients_model=[]
        clients_loss =[]

        for k in range(no_clients):

          def objective_function(solution):

            lr = solution[0]
            momentum = solution[1]
            decay = solution[2]
            local_model = deepcopy(model)
            local_optimizer=optim.SGD(local_model.parameters(),lr=lr,momentum=momentum,weight_decay=decay)
            local_loss=local_learning(local_model,local_optimizer,train[k],local_epoch,loss_f)

            clients_loss.append(local_loss)
            list_param=1
             #GET THE PARAMETER TENSORS OF THE MODEL
            list_params=list(local_model.parameters())
            list_params=[tens_param.detach() for tens_param in list_params]
            clients_param.append(list_params)
            clients_model.append(deepcopy(local_model))

            return local_loss


          model_hyper = OriginalAVOA(epoch=tuning_epoch,pop_size=100,p1=0.6, p2=0.4, p3=0.6, alpha=0.8, gama=2.5)

          problem_iid = {
                    "bounds": FloatVar(lb=(0.01,0.001,0.001),ub=(0.1,0.1,0.1), name="delta"),
                    "obj_func":objective_function,
                    "minmax": "min",
          }


          model_hyper.solve(problem_iid)


          min_loss = min(clients_loss)
          min_index = clients_loss.index(min_loss)

          clients_params.append(clients_param[min_index])
          clients_models.append(clients_model[min_index])
          clients_losses.append(clients_loss[min_index])

          clients_param=[]
          clients_model=[]
          clients_loss =[]

          print(f"Best Learning Rate:{model_hyper.solution[0][0]},Best momentum Rate:{model_hyper.solution[0][1]},Best Decay Rate:{model_hyper.solution[0][2]}")
          print(f"Lowest Loss:{model_hyper.solution[0]},Lowest:{model_hyper.solution[1]}")

          models_hist.append(clients_models)

          loss_hist+=[[float(loss_dataset(model, dl, loss_f).detach())
                    for dl in train]]
          acc_hist+=[[accuracy_dataset(model, dl) for dl in test]]

          server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
          server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])

          print(f'====> i: {i+1} Loss: {server_loss} Server Test Accuracy: {server_acc}')
          server_accuracy_list.append(server_acc)
          server_loss_list.append(server_loss)


          server_hist.append([tens_param.detach().cpu().numpy()
                    for tens_param in list(model.parameters())])

In [ ]:
train,test = MNIST()

#for client_id, train_loader in enumerate(train):
    #print(f"Client {client_id + 1} - Training Data:")
    #for batch_idx, (data, target) in enumerate(train_loader):
       #print(f"Batch {batch_idx + 1} - Data Shape: {data.shape}, Target Shape: {target.shape}")

model = MNISTCNN()
#temp variables and can be altered
n_iter= 10
local_epoch = 100
tuning_epoch = 50


FEDAVO(train,test,model,n_iter,local_epoch,tuning_epoch)

Clients' weights: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
====> i: 0 Loss: 2.3037845293680825


  0%|          | 0/10 [00:00<?, ?it/s]INFO:mealpy.swarm_based.AVOA.OriginalAVOA:Solving single objective optimization problem.
INFO:mealpy.swarm_based.AVOA.OriginalAVOA:>>>Problem: P, Epoch: 1, Current best: 18.378416061401367, Global best: 18.378416061401367, Runtime: 18.79334 seconds
INFO:mealpy.swarm_based.AVOA.OriginalAVOA:>>>Problem: P, Epoch: 2, Current best: 18.39268684387207, Global best: 18.378416061401367, Runtime: 22.77802 seconds
INFO:mealpy.swarm_based.AVOA.OriginalAVOA:>>>Problem: P, Epoch: 3, Current best: 18.38743782043457, Global best: 18.378416061401367, Runtime: 18.44589 seconds
INFO:mealpy.swarm_based.AVOA.OriginalAVOA:>>>Problem: P, Epoch: 4, Current best: 18.3868465423584, Global best: 18.378416061401367, Runtime: 18.48484 seconds
  0%|          | 0/10 [01:53<?, ?it/s]


KeyboardInterrupt: 

In [ ]:

def __init__():

  main()

In [ ]:
# CHIRAG
from mealpy.swarm_based.PSO import AIW_PSO
import numpy as np
from mealpy import FloatVar, PSO
import tensorflow as tf
from tensorflow.keras.losses import mean_squared_error

# Define the SGD loss function
def sgd_loss(y_true, y_pred):
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

# Define the objective function (negative of the SGD loss)
def objective_function(solution):
    # Reshape the solution to match the neural network parameters
   # neural_network_params = solution.reshape((input_size, hidden_size, output_size))

    # Build and compile a simple neural network model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_size,)),
        tf.keras.layers.Dense(1, activation='linear')
    ])

    #model.set_weights(neural_network_params)
    model.compile(optimizer='sgd', loss=sgd_loss)

    # Generate random input data
    x = np.random.rand(num_samples, input_size)
    # Generate random ground truth labels
    y_true = np.random.rand(num_samples, output_size)

    # Use the model to predict and calculate the negative of the SGD loss
    y_pred = model.predict(x)
    loss = -sgd_loss(y_true, y_pred).numpy()

    return loss

# Define the problem dictionary
problem_dict = {
    "obj_func": objective_function,
    "bounds": FloatVar( lb=(-10.,) * 30, ub=(10.,) * 30, name="delta"),
    # "lb": -1.0,  # Lower bound for each parameter
    # "ub": 1.0,   # Upper bound for each parameter
   # "dim": (input_size * hidden_size * output_size),  # Total number of parameters
    "constraints": None,
    "work_dir": None,
    "epoch": [100,1000],  # Number of iterations/epochs
    "pop_size": 50,  # Population size
    "w_max": 0.9,   # Maximum inertia weight
    "w_min": 0.2,   # Minimum inertia weight
    "c1": 1.5,      # Cognitive parameter
    "c2": 1.5,      # Social parameter
    "solution": None,
    "eval_flag": True,
    "flag": 'Min'  # Indicates that it is a minimization problem
}

# Create and solve the PSO problem
optimizer = AIW_PSO(epoch=100, pop_size=50, c1=2.05, c2=3.5, alpha=0.4)
best_solution, best_fitness = optimizer.solve(problem_dict)

print("Best Solution:", best_solution)
print("Best Fitness (Negative SGD Loss):", -best_fitness)


NameError: name 'input_size' is not defined